## Filter non human

This job to filter non human unmapped reads from BAM file via samtools. It needs a BED file to filter specific reads regions.

In [ ]:
import traceback
import pysam
import os
from hops import hdfs
import utils
import sys
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

#### Load arguments

In [ ]:

args_full=utils.load_arguments(sys.argv)

OUTPUT_DATASET=args_full[utils.OUTPUT_DATASET]
INPUT_ROOT_PATH=args_full[utils.INPUT_ROOT_PATH]
RUN_FOLDER=args_full[utils.RUN_FOLDER]
WORK_PATH=os.path.join(OUTPUT_DATASET, RUN_FOLDER)
args=args_full[utils.KEY_REMOVE_HUMAN]

# check of input and output root override
if args_full.get(utils.INPUT_OVERRIDE):
    inputRoot=args_full.get(utils.INPUT_OVERRIDE)
else :
    inputRoot=os.path.join(WORK_PATH,args['INPUT_ROOT'])
if args_full.get(utils.OUTPUT_OVERRIDE):
    outputBam=args_full.get(utils.OUTPUT_OVERRIDE)
else:
    outputBam=os.path.join(WORK_PATH,args['OUTPUT_BAM'])


humanFilterPath=args['FILTER_BED']

threads=str(args['THREADS'])


#### map function

In [ ]:

def remove_human(file):
    """

    Runs pysam on a BAM file to filter specific reads region as specified in BED file.
    Output is copied back to hdfs.
    If output file name is already present in destination the process is skipped
    to avoid processing of same file incase of resubmit of failed run.

    :param file:
    :return:
    """
    
    filename=os.path.basename(file)
    bam_file=filename.split('.')[0]+'_NH.bam'
    if not hdfs.exists(os.path.join(outputBam,bam_file)): # check if output already exists
    
        hdfs.copy_to_local(humanFilterPath)
        humanFilter=os.path.basename(humanFilterPath)
        hdfs.copy_to_local(file, overwrite=True)        
        print("INFO: Run non human BAM : ", filename)
        try:
            pysam.view('-o','/dev/null', '-L', humanFilter, '-U',bam_file, filename,'-@',threads, catch_stdout=False)
            if os.path.exists(bam_file):
                hdfs.copy_to_hdfs(bam_file,outputBam,overwrite=True)
                os.remove(bam_file)
        except pysam.SamtoolsError:
            traceback.print_exc()
        finally:
            os.remove(filename)

        return bam_file
    
    else:
        print('skipping existing file: ', filename)            
        return None




    


#### Load input files hdfs paths

In [ ]:

inputFiles=utils.load_file_names(inputRoot)



In [ ]:
print('Number of input files: ', len(inputFiles))

#### Run in parallel

In [ ]:

### make spark rdd and map function
unMapped=sc.parallelize(inputFiles,sc.getConf().get("spark.executor.instances")).map(remove_human).collect()
